## Graphics 
### Correlation Matrices for a year

In [2]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

corr=pd.read_csv("corr_2008.csv")
corr=corr[corr.columns[1:]]
labels=corr.columns.values

In [3]:
data=[go.Heatmap(z=corr.values.tolist(),colorscale='RdYlGn', x=labels, y=labels)]
#Add title to plot
#Save the plot. 

py.iplot(data, filename='pandas-heatmap')


### Production of Big_Data Matrix

In [111]:
from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as FF 
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML
import os
import pandas as pd
import re 
import numpy as np


#Concatenate all years into one big data_frame:
files= os.listdir(".")
big_data=pd.DataFrame()
for f in files:
    if f.startswith("price_"):
        data=pd.read_csv(f)
        # #extract year from filename using regex
        p=re.compile('price_(\d+).csv')
        m=p.match(f)
        year=m.group(1)
        #Append data for all years with extra column of years.
        year_vector=[year]*data.shape[0]
        data['year']=year_vector
        big_data=big_data.append(data)
        big_data.to_csv("all_price_data")
        

#Get information of types of company stocks
info=pd.read_csv("sp500.csv")

#Get set of sectors and years for graphic
sectors=list(set(info["sectors"]))
years=[i for i in reversed(list(set(big_data["year"])))]

#Read information of volumes for all years
vol_data=pd.read_csv("all_vol_data")

#### Create data of mean stock value per year 
mean_data=big_data.groupby('year').mean().transpose()
mean_vol=vol_data.groupby('year').mean().transpose()
mean_data['sector']=info['sectors'].values
mean_data['company']=mean_data.index.values

#Reformat the data to fit the requirement to do the dynamic dataset
SP=pd.concat([pd.melt(mean_data,var_name="years",id_vars=['sector','company'], value_name="stock_price"),
           pd.melt(mean_vol,var_name="years", value_name="mean_volume")["mean_volume"]]
          ,axis=1)


SP['mean_volume']=np.sqrt(SP['mean_volume'])

### Static Visualization of Stock Information

In [5]:
#Interest in Stock Comparison for Apple, Google, and Yahoo for all the years
import plotly.plotly as py
import plotly.graph_objs as go


trace=go.Scatter(x=range(0,big_data.shape[0],1), 
                 y=big_data["GOOGL"].values,
                 mode='markers',
                 name='Google')

trace_1=go.Scatter(x=range(0,big_data.shape[0],1), 
                 y=big_data["AAPL"].values,
                 mode='markers',
                 name='Apple')
trace_2=go.Scatter(x=range(0,big_data.shape[0],1), 
                 y=big_data["YHOO"].values,
                 mode='markers',
                 name='Yahoo')

#Set the layout: axis labels, titles, etc.
layout=go.Layout(title='Stock Behavior for Info and Tech Companies 2008-2016',
                yaxis=dict(
                title='Stock_Value',
                titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
        )
    ))

fig=go.Figure(data=[trace,trace_1, trace_2],layout=layout)
py.iplot(fig, filename='google-stock_scatter')




In [5]:
import plotly.plotly as py
import plotly.graph_objs as go


trace=go.Scatter(x=range(0,big_data.shape[0],1), 
                 y=big_data["CVX"].values,
                 mode='markers',
                 name='Chevron')

trace_1=go.Scatter(x=range(0,big_data.shape[0],1), 
                 y=big_data["DVN"].values,
                 mode='markers',
                 name='Devon')
trace_2=go.Scatter(x=range(0,big_data.shape[0],1), 
                 y=big_data["XOM"].values,
                 mode='markers',
                 name='Exon')

#Set the layout: axis labels, titles, etc.
layout=go.Layout(title='Stock Behavior for Energy Companies 2008-2016',
                yaxis=dict(
                title='Stock_Value',
                titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
        )
    ))

fig=go.Figure(data=[trace,trace_1, trace_2],layout=layout)
py.iplot(fig, filename='energy-stock_scatter')



### Dynamic Visualization of Stock Information

In this section we attempt to do a motion of dataframes to see how the stocks changed over the years. Since the dataset is too big, we will do some subsetting to only keep the companies that had a greater change from 2008 to 2016. 

In [114]:
#Data Prep: Since Data is too big to plot then we ca subset the dataset to keep
rel_mean_change=(mean_data['2016']-mean_data['2008'])/mean_data['2008']
mean_data=mean_data[rel_mean_change > 1.5]

#Subset for those with bigger relative change
SP=SP[SP['company'].isin(mean_data.index.values)]
#Remove those companies with missing values in mean_volume and stock-price
SP=SP.dropna()





In [115]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML
init_notebook_mode(connected=True)
from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as FF 


#make figure 
figure = {
    'data': [],
    'layout': {},
    'frames': [],
    'config': {'scrollzoom': True}
}

#fill layout
figure['layout']['xaxis']={'title':'Sqrt(Mean Volume)','range':[min(SP['mean_volume']),max(SP['mean_volume'])] }
figure['layout']['yaxis'] = {'title': 'Mean Stock Value'}
figure['layout']['hovermode'] = 'closest'

#Slider information
figure['layout']['slider'] = {
    'args': [
        'slider.value', {
            'duration': 400,
            'ease': 'cubic-in-out'
        }
    ],
    'initialValue': '2008',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

# determine where on the plot the slider will appear.
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

#make data
year = '2008'
for sector in sectors:
    dataset_by_year = SP[SP["years"]==year]
    dataset_by_year_and_sector = dataset_by_year[dataset_by_year['sector']==sector]
        
    data_dict = {
            'x':list(dataset_by_year_and_sector['mean_volume']),
            'y': list(dataset_by_year_and_sector['stock_price']),
            'text': list(dataset_by_year_and_sector['company']), #this should be stock name 
            'mode':'markers',
            'marker': {
                'sizemode': 'area',
                'sizeref': 200000,
            },
            'name': sector
        }
        
    figure['data'].append(data_dict)

    
# make frames

for year in years:
    frame = {'data': [], 'name': str(year)}
    for sector in sectors:
        dataset_by_year = SP[SP['years']==year]
        dataset_by_year_and_sector = dataset_by_year[dataset_by_year['sector']==sector]
        
        data_dict = {
            'x':list(dataset_by_year_and_sector['mean_volume']),
            'y': list(dataset_by_year_and_sector['stock_price']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_sector['company']),  
            'marker': {
                'sizemode': 'area',
                'sizeref': 200000,
            },
            'name': sector
        }
        frame['data'].append(data_dict)
        
    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 500, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 500}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

    
figure['layout']['sliders'] = [sliders_dict]
iplot(figure, validate=False)
                                                   